<a href="https://colab.research.google.com/github/LikaOkudzhava/BrainyScan/blob/InceptionV3/Alzheimer_model_inceptionV3_version3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from tqdm import tqdm

import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

In [ ]:
import os

import zipfile

from google.colab import drive

gdrive = '/content/drive'

gdisk = os.path.join(gdrive, 'MyDrive')

filename = 'smallpreprocessed'

archive_path = os.path.join(gdisk, filename)

with zipfile.ZipFile('/content/drive/MyDrive/AI/smallpreprocessed.zip', 'r') as zip_ref:

 zip_ref.extractall(f'{filename}')

In [ ]:
dataset_dir = '/content/smallpreprocessed/data/SmallPreprocessed'
image_height = 299
image_width = 299
batch_size = 16


train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(dataset_dir, 'train'),
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = (image_height, image_width),
    batch_size = batch_size,
    shuffle = True
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(dataset_dir, 'test'),
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = (image_height, image_width),
    batch_size = batch_size,
    shuffle = False
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(dataset_dir, 'val'),
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = (image_height, image_width),
    batch_size = batch_size,
    shuffle = True
)

for ds in [train_ds, test_ds, val_ds]:
    ds.cache()
    ds.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 12000 files belonging to 4 classes.
Found 2000 files belonging to 4 classes.
Found 2000 files belonging to 4 classes.


In [ ]:
from tensorflow import keras

# Load InceptionV3 with all layers frozen
model = keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(299, 299, 3)
)

# Freeze all layers
for layer in model.layers:
    layer.trainable = False

In [ ]:
from tensorflow import keras
import tensorflow as tf

# Image dimensions
image_height = 299
image_width = 299
input_shape = (image_height, image_width, 3)

# Input layer
inputs = keras.Input(shape=input_shape)

# Skip augmentation — direct input
x = inputs

# Load InceptionV3 base model with pretrained weights
base_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    pooling='avg'  # Outputs a global average pooled vector
)
base_model.trainable = False  # Freeze pretrained weights

# Pass input through base model
x = base_model(x)

# Custom classification head
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
outputs = keras.layers.Dense(4, activation='softmax')(x)  # 4 classes

# Combine into a full model
model = keras.Model(inputs=inputs, outputs=outputs)

# Show model summary
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_26 (InputLayer)     │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_v3 (Functional)       │ (None, 2048)           │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1332        │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1333        │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1334        │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 4)              │         4,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,121,892 (107.28 MB)

 Trainable params: 6,308,868 (24.07 MB)

 Non-trainable params: 21,813,024 (83.21 MB)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 74s 78ms/step - accuracy: 0.4589 - loss: 1.5624 - val_accuracy: 0.6070 - val_loss: 0.9738
Epoch 2/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 52s 69ms/step - accuracy: 0.5899 - loss: 0.9682 - val_accuracy: 0.6520 - val_loss: 0.8607
Epoch 3/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 86s 74ms/step - accuracy: 0.6446 - loss: 0.8622 - val_accuracy: 0.6915 - val_loss: 0.7924
Epoch 4/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 55s 74ms/step - accuracy: 0.6869 - loss: 0.7571 - val_accuracy: 0.7295 - val_loss: 0.6344
Epoch 5/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 55s 73ms/step - accuracy: 0.7141 - loss: 0.6978 - val_accuracy: 0.7380 - val_loss: 0.6896
Epoch 6/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 79s 70ms/step - accuracy: 0.7480 - loss: 0.6273 - val_accuracy: 0.7330 - val_loss: 0.7519
Epoch 7/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 52s 69ms/step - accuracy: 0.7630 - loss: 0.5869 - val_accuracy: 0.7745 - val_loss: 0.5796
Epoch 8/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 70ms/step - accuracy: 0.7888 - loss: 0.5328 - 

In [ ]:
for layer in model.layers[-50:]:
    layer.trainable = True

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Model Checkpoint - save only the best model
checkpoint_cb = ModelCheckpoint(
    "model_InceptionV3.keras",  # Updated filename
    save_best_only=True
)

# Early Stopping - stop if no improvement for 10 epochs
early_stopping_cb = EarlyStopping(
    patience=10,
    restore_best_weights=True
)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
EPOCHS = 25

In [ ]:
# Now train with class weights
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[checkpoint_cb, early_stopping_cb],
    # class_weight=class_weights
)

Epoch 1/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 259s 249ms/step - accuracy: 0.5912 - loss: 1.4650 - val_accuracy: 0.7260 - val_loss: 0.7373
Epoch 2/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 187s 249ms/step - accuracy: 0.8363 - loss: 0.4153 - val_accuracy: 0.8825 - val_loss: 0.2963
Epoch 3/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 177s 236ms/step - accuracy: 0.9264 - loss: 0.1887 - val_accuracy: 0.8925 - val_loss: 0.3042
Epoch 4/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 207s 243ms/step - accuracy: 0.9528 - loss: 0.1301 - val_accuracy: 0.9005 - val_loss: 0.2845
Epoch 5/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 184s 245ms/step - accuracy: 0.9628 - loss: 0.1075 - val_accuracy: 0.9435 - val_loss: 0.1726
Epoch 6/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 177s 236ms/step - accuracy: 0.9653 - loss: 0.1014 - val_accuracy: 0.9295 - val_loss: 0.1991
Epoch 7/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 204s 240ms/step - accuracy: 0.9735 - loss: 0.0742 - val_accuracy: 0.9090 - val_loss: 0.2855
Epoch 8/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 186s 248ms/step - accuracy: 0.9790 -

In [ ]:
model.save("/content/drive/MyDrive/AI/inception_model.h5")


In [ ]:
model.save('/content/model_InceptionV3.keras')
model.export('/content/models/inceptionV3_model')

In [ ]:
hist_=pd.DataFrame(history.history)

In [ ]:
score, acc= model.evaluate(test_ds)
print('Val Loss =', score)
print('Val Accuracy =', acc)

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(1,2,1)
plt.plot(hist_['loss'],label='Train_Loss')
plt.plot(hist_['val_loss'],label='Validation_Loss')
plt.title('Train_Loss & Validation_Loss',fontsize=20)
plt.legend()
plt.subplot(1,2,2)
plt.plot(hist_['accuracy'],label='Train_Accuracy')
plt.plot(hist_['val_accuracy'],label='Validation_Accuracy')
plt.title('Train_Accuracy & Validation_Accuracy',fontsize=20)
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd

# Convert test labels (one-hot) to class indices
y_test_labels = []
for batch in test_ds:
    images, labels = batch
    y_test_labels.append(labels.numpy())

y_test = np.concatenate(y_test_labels, axis=0)
y_test = np.argmax(y_test, axis=1)  # Convert one-hot to class index

# Predict using model
predictions = model.predict(test_ds)
y_pred = np.argmax(predictions, axis=1)  # Class index

# Create a DataFrame with Actual vs Prediction
df = pd.DataFrame({'Actual': y_test, 'Prediction': y_pred})


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Ensure your predicted and true labels are available
# y_test and y_pred should be 1D arrays of class indices

# Define class labels (update to match your actual class names)
class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

# Compute the confusion matrix
CM = confusion_matrix(y_test, y_pred)

# Normalize the confusion matrix
CM_percent = CM.astype('float') / CM.sum(axis=1)[:, np.newaxis]

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(CM_percent,
            fmt='.2f',
            annot=True,
            cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            cbar=False)

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Normalized Confusion Matrix')
plt.show()

# Optionally print the raw CM
CM


In [ ]:
ClassificationReport = classification_report(y_test,y_pred,target_names=class_names)
print('Classification Report is : ', ClassificationReport)